In [6]:
import os
import json
import pandas as pd

In [12]:
data_dir = './data_dump'
pd.set_option('display.max_columns', None)

In [13]:
for fn in os.listdir(data_dir):
  with open(os.path.join(data_dir, fn), 'r') as f:
    data = json.load(f)
    df_raw = pd.json_normalize(data)

In [ ]:
static_data = {}
static_cols = ['angular_velocity', 'position', 'velocity', 'rotation']
for col in df_raw.columns:
  if len(df_raw[col].unique()) == 1 and all(stc not in col for stc in static_cols):
    static_data[col] = df_raw[col].unique()[0]
static_data

{'medium.base_density': np.float64(1.225),
 'medium.drag_coef': np.float64(0.47),
 'medium.drag_model': 'quadratic',
 'medium.molar_mass': np.float64(0.029),
 'medium.rha_coef': np.float64(1e-05),
 'medium.speed_of_sound': np.float64(343.0),
 'medium.type': 'gas',
 'medium.viscosity': np.float64(1.81e-05),
 'naming.ammo_name': '5.56x45_M855',
 'naming.projectile_uid': 'proj_ZAuKc4a2',
 'naming.weapon_name': 'SCAR_L_CQC',
 'projectile.ammo': '<Resource#-9223371996220357260>',
 'projectile.caliber': np.float64(5.7),
 'projectile.core_caliber': np.float64(5.4),
 'projectile.core_hardness': np.float64(0.6),
 'projectile.core_mass': np.float64(2.0),
 'projectile.cross_section': np.float64(2.55175863287831e-05),
 'projectile.drag_coef': np.float64(0.28),
 'projectile.fragmentation_chance': np.float64(0.1),
 'projectile.fragmentation_count': np.int64(0),
 'projectile.fragments_max': np.int64(4),
 'projectile.fragments_min': np.int64(2),
 'projectile.impact_count': np.int64(0),
 'projectile.ki